In [127]:
import numpy as np
from lsst.obs.lsst.cameraTransforms import LsstCameraTransforms
from lsst.obs.lsst import LsstCamMapper as camMapper
%matplotlib ipympl

In [138]:
raft = 'R10'
sensor = 'S01'
reb = 'Reb1'
ccdid = raft + '_' + sensor
camera = camMapper._makeCamera()
lct = LsstCameraTransforms(camera)
det = lct.getDetector(ccdid)
det.getSerial()

'ITL-3800C-040'

### Define the center of the scan as the center of Segment 14 of the central CCD
Finding where the center of Segment 14 of the CCD is in focal plane coordinates.
Note that (xcam, ycam) = (yfp, xfp) because of the different conventions

In [151]:
ycenter, xcenter = lct.ccdPixelToFocalMm(2304,3003,ccdid)
#ycenter, xcenter = lct.ccdPixelToFocalMm(0,0,ccdid)

In [152]:
xcenter, ycenter

(-158.885, -251.80500000000004)

### Define the scan parameters: Delta_x, Delta_y, N_x, N_y
From now on, working in camera coordinate system

In [141]:
Delta_x = 80 # mm
Delta_y = 80 # mm
N_x = 12
N_y = 12

In [142]:
xmin = xcenter - Delta_x / 2.
xmax = xcenter + Delta_x / 2.
ymin = ycenter - Delta_y / 2.
ymax = ycenter + Delta_y / 2.
print(xmin, xmax, ymin, ymax)

-198.885 -118.88499999999999 -291.80500000000006 -211.80500000000004


In [143]:
xarr = np.linspace(xmin, xmax, N_x)
yarr = np.linspace(ymin, ymax, N_y)
#print(xarr, yarr)

In [144]:
xall = np.repeat(xarr, N_y)
yall = np.broadcast_to(yarr, (N_y, N_x)).flatten()

In [145]:
led = ['uv', 'blue', 'red', 'nm750', 'nm850', 'nm960']
current = [0.1, 0.006, 0.009, 0.01, 0.01, 0.01]
exp_time = [0.1, 0.25, 0.25, 0.45, 0.3, 0.6]
led_config = list(zip(led, current, exp_time))
led_config

[('uv', 0.1, 0.1),
 ('blue', 0.006, 0.25),
 ('red', 0.009, 0.25),
 ('nm750', 0.01, 0.45),
 ('nm850', 0.01, 0.3),
 ('nm960', 0.01, 0.6)]

In [146]:
led_number = 5
np.savetxt('tmp.txt', list(zip(xall, yall)),fmt='%.3f', delimiter=('  '))
config_filename = 'ccob_'+str(N_x)+'x'+str(N_y)+'_'+raft+'_'+sensor+'_'+led[led_number]+'.cfg'
config_filename

'ccob_12x12_R10_S01_nm960.cfg'

In [147]:
with open('tmp.txt', 'r') as istr:
    with open(config_filename, 'w') as ostr:
        print('[ACQUIRE]', file=ostr)
        print('bias', file=ostr)
        print('ccob', file=ostr)
        print('bias\n', file=ostr)
        print('[BIAS]', file=ostr)
        print('COUNT=10\n', file=ostr)
        print('[CCOB]', file=ostr)
        print('BCOUNT = 0', file=ostr)
        print('IMCOUNT = 1', file=ostr)
        print('XOFFSET = 0', file=ostr)
        print('YOFFSET = 0\n', file=ostr)
        print('LOCATIONS = '+raft+'/'+reb+'\n', file=ostr)
        print('expose = '+ str(led_config[led_number][0])+' '
              + str(led_config[led_number][1])+' '+str(led_config[led_number][2])+'\n', file=ostr)
        for i,line in enumerate(istr):
            if i==0: 
                line = 'point =  ' + line.rstrip('\n') + ','
            elif i!=len(xall)-1:
                line = '         ' + line.rstrip('\n') + ','
            else:
                line = '         ' + line.rstrip('\n')
            print(line, file=ostr)